In [1]:
from pathlib import Path
import pandas as pd 
from sqlalchemy import create_engine
import os

In [2]:
BASE_DIR = Path().cwd().parent
DATA_DIR = BASE_DIR / "datasets" / "productos_clasificados.csv"

df = pd.read_csv(DATA_DIR)
df['product_name']

0                                  Néctar de durazno 1 L
1                                Sardinas en salsa 155 g
2                                      Queso gouda 300 g
3                                      Frijol pinto 1 kg
4                                   Huevo blanco 12 pzas
                             ...                        
223                  Plato unicel con divisiones 15 pzas
224                    Charola unicel para carne 50 pzas
225    Contenedor térmico unicel con tapa (Hamburgues...
226              Vaso unicel térmico 1 L (Litro) 10 pzas
227                    Tapa para vaso unicel 1 L 50 pzas
Name: product_name, Length: 228, dtype: object

In [3]:
import pandas as pd

# Creamos un patrón que dice:
# 1. (\d+(\.\d+)?) -> Busca números (enteros o decimales)
# 2. \s* -> Seguido de un espacio opcional
# 3. (kg|g|pzas|L|ml|oz) -> Seguido de una de estas unidades
# 4. $             -> Y que eso esté AL FINAL del texto

patron = r'\s\d+(\.\d+)?\s*(kg|g|pzas|pz|L|ml|oz|)$'

# Reemplazamos ese patrón por "" (nada)
# case=False hace que le de igual si es Kg, KG o kg
# regex=True activa el modo "búsqueda de patrones"
df['product_name'] = df['product_name'].str.replace(patron, '', case=False, regex=True)

# Limpieza final: .strip() quita los espacios sobrantes a los lados
df['product_name'] = df['product_name'].str.strip()

print(df['product_name'].tail(10))

218                          Leche Condensada La Lechera
219                                Encendedor desechable
220                         Plato unicel pastelero No. 6
221                      Plato unicel plano grande No. 9
222                          Plato unicel hondo pozolero
223                          Plato unicel con divisiones
224                            Charola unicel para carne
225    Contenedor térmico unicel con tapa (Hamburgues...
226                      Vaso unicel térmico 1 L (Litro)
227                            Tapa para vaso unicel 1 L
Name: product_name, dtype: object


In [4]:
df = df[['product_name', 'category_off', 'shelf_life_pantry_days', 'shelf_life_fridge_days','shelf_life_freezer_days', 'perecedero']]
df = df.drop_duplicates(subset=['product_name'], keep='first')

In [5]:
user = "root"
password = "root"
host = "localhost"
db_name = "pt"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db_name}")


df.to_sql(
    name='products', #Nombre de la tabla en mysql
    con=engine,
    if_exists='append', 
    index=False
)

96